In [ ]:
from evo.aio import AioTransport
from evo.common import APIConnector
from evo.oauth import AuthorizationCodeAuthorizer, EvoScopes, OAuthConnector
from evo.workspaces import WorkspaceAPIClient

cache_location = "./notebook-data"
input_path = f"{cache_location}/input"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL
org_id = "<your-organization-id>"  # Replace with your organization ID
hub_url = "<your-hub-url>"  # Replace with your Evo hub URL

client_id = "daves-evo-client"
redirect_url = "http://localhost:32369/auth/callback"
org_id = "fab9774b-b338-4cc2-a6c9-458bdf7f966a"  # Replace with your organization ID
hub_url = "https://au.api.seequent.com"  # Replace with your Evo hub URL

client_name = "Evo Python SDK Notebook Example"

transport = AioTransport(
    user_agent=client_name,
)

connector = OAuthConnector(
    transport=transport,
    client_id=client_id,
)

authorizer = AuthorizationCodeAuthorizer(
    oauth_connector=connector,
    redirect_url=redirect_url,
    scopes=EvoScopes.all_evo | EvoScopes.offline_access,
)

await authorizer.login()

### Prepare Workspace API client

In [ ]:
workspace_client = WorkspaceAPIClient(
    connector=APIConnector(hub_url, transport, authorizer),
    org_id=org_id,
)

## Workspace operations

### List all workspaces

In [ ]:
import pandas as pd

try:
    all_workspaces = await workspace_client.list_all_workspaces()

    # Display the workspaces in a table
    workspace_list = [
        {
            "Name": workspace.display_name,
            "ID": workspace.id,
            "User Role": workspace.user_role,
        }
        for workspace in all_workspaces
    ]

    df = pd.DataFrame(workspace_list)
    display(df)
except Exception as e:
    print(f"Error listing workspaces:\n{e}")

### Create a workspace

In [ ]:
try:
    new_workspace = await workspace_client.create_workspace(
        name="My new 5workspace",
        description="This is a new workspace",
        bounding_box_coordinates=[
            (85.8287, -90.0),
            (85.8287, 69.84278),
            (-180.0, 69.84278),
            (-180.0, -90.0),
            (85.8287, -90.0),
        ],
        # A square bounding box. The first and last coordinates must be the same value.
        # Also, the first coordinate element must be longitude and the second must be latitude.
    )

    # Display the workspace details
    workspace_info = {
        "Name": new_workspace.display_name,
        "ID": new_workspace.id,
        "Description": new_workspace.description,
        "User Role": new_workspace.user_role,
        "Created At": new_workspace.created_at,
    }

    df = pd.DataFrame([workspace_info]).T
    df.columns = ["Workspace info"]
    display(df)
except Exception as e:
    print(f"Error creating workspace: {e}")

### Fetch details for a specific workspace

In [ ]:
import pandas as pd

try:
    workspace = await workspace_client.get_workspace(workspace_id=new_workspace.id)

    # Display the workspace details
    workspace_info = {
        "Name": workspace.display_name,
        "ID": workspace.id,
        "User Role": workspace.user_role,
    }

    df = pd.DataFrame([workspace_info]).T
    df.columns = ["Workspace info"]
    display(df)
except Exception as e:
    print(f"Error listing workspaces:\n{e}")

### Update a workspace

In [ ]:
try:
    updated_workspace = await workspace_client.update_workspace(
        workspace_id=new_workspace.id,
        name="My updated4 workspace",
        description="This is an updated workspace",
    )

    # Display the updated workspace details
    workspace_info = {
        "Name": updated_workspace.display_name,
        "ID": updated_workspace.id,
        "Description": updated_workspace.description,
        "User Role": updated_workspace.user_role,
        "Created At": updated_workspace.created_at,
    }

    df = pd.DataFrame([workspace_info]).T
    df.columns = ["Workspace info"]
    display(df)
except Exception as e:
    print(f"Error updating workspace: {e}")

### Delete a workspace

When you delete a workspace, it will be soft-deleted. Deleted workspaces will no longer be accessible through `list_all_workspaces` by default.
The `delete_workspace` function has no return value.

In [ ]:
try:
    await workspace_client.delete_workspace(workspace_id=new_workspace.id)
    print(f"Successfully deleted workspace: {new_workspace.display_name}")
except Exception as e:
    print(f"Error deleting workspace: {e}")

### Fetch details of a deleted workspace

You must include the `deleted=True` option in find details of a deleted workspace.

In [ ]:
import pandas as pd

try:
    workspace = await workspace_client.get_workspace(workspace_id=new_workspace.id, deleted=True)

    # Display the workspace details
    workspace_info = {
        "Name": workspace.display_name,
        "ID": workspace.id,
        "User Role": workspace.user_role,
    }

    df = pd.DataFrame([workspace_info]).T
    df.columns = ["Workspace info"]
    display(df)
except Exception as e:
    print(f"Error listing workspaces:\n{e}")

### Restore a deleted workspace

To restore a deleted workspace, see the code sample in `api-examples.ipynb`.

## User operations

Users can be added to a workspace with one of three user roles:
- Owner
- Editor
- Viewer

Workspace roles follow these rules:
- Users who have the owner role in a workspace can add users with any one of the three roles.
- Users who have the editor role in a workspace can add users with the either the editor or the viewer role.
- Users who have the viewer role can't add other users to a workspace.

To change a user's role in a workspace you must know both the ID of the workspace and the ID of the user.

### Fetch details of a user's workspace role

In [ ]:
from evo.workspaces import UserRole

try:
    user_role: UserRole = await workspace_client.get_current_user_role(workspace_id=new_workspace.id)
except Exception as e:
    print(f"Error retrieving user role:\n{e}")

print(f"Current user's role in workspace '{new_workspace.display_name}': {user_role.role}")

### List users with access to a workspace

In [ ]:
from evo.workspaces import User

try:
    users: list[User] = await workspace_client.list_user_roles(workspace_id=new_workspace.id)

    # Display the users details
    user_roles_list = [
        {"Full name": user.full_name, "Email": user.email, "Role": user.role, "User ID": user.user_id} for user in users
    ]

    df = pd.DataFrame(user_roles_list)
    display(df)
except Exception as e:
    print(f"Error listing users in this workspace:\n{e}")

### Assign a user to a workspace

In [ ]:
import uuid

from evo.workspaces import WorkspaceRole

try:
    new_users_role: UserRole = await workspace_client.assign_user_role(
        workspace_id=new_workspace.id, user_id=uuid.UUID("<some-user-id>"), role=WorkspaceRole.viewer
    )
except Exception as e:
    print(f"Error assigning user role:\n{e}")

### Remove a user from a workspace

In [ ]:
import uuid

try:
    await workspace_client.delete_user_role(
        workspace_id=new_workspace.id,
        user_id=uuid.UUID("<some-user-id>"),
    )
except Exception as e:
    print(f"Error removing user role:\n{e}")